In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
code = """
#include <stdio.h>
#include <stdlib.h>

#define N 1024
#define BLOCK_SIZE 16

__global__ void matrixMul(int *a, int *b, int *c, int width) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    for (int i = 0; i < width; i++) {
        sum += a[row * width + i] * b[i * width + col];
    }
    c[row * width + col] = sum;
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * N * sizeof(int);

    // Allocate memory on host
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize matrices
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            a[i * N + j] = i + j;
            b[i * N + j] = i - j;
        }
    }

    // Allocate memory on device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 2D grid and 2D block
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 dimGrid((N + dimBlock.x - 1) / dimBlock.x, (N + dimBlock.y - 1) / dimBlock.y);
    matrixMul<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, N);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print first and last elements of result
    printf("c[0][0] = %d, c[%d][%d] = %d", c[0], N-1, N-1, c[(N-1) * N + (N-1)]);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

"""

In [3]:
text_file=open("assign4b.cu","w")
text_file.write(code)
text_file.close()

In [4]:
!nvcc assign4b.cu

In [5]:
!./a.out

c[0][0] = 357389824, c[1023][1023] = -714255872

In [6]:
!nvprof ./a.out

==707== NVPROF is profiling process 707, command: ./a.out
==707== Profiling application: ./a.out
c[0][0] = 357389824, c[1023][1023] = -714255872==707== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   75.07%  9.0869ms         1  9.0869ms  9.0869ms  9.0869ms  matrixMul(int*, int*, int*, int)
                   12.78%  1.5469ms         2  773.47us  743.10us  803.83us  [CUDA memcpy HtoD]
                   12.15%  1.4709ms         1  1.4709ms  1.4709ms  1.4709ms  [CUDA memcpy DtoH]
      API calls:   93.11%  210.38ms         3  70.126ms  68.256us  210.24ms  cudaMalloc
                    6.12%  13.827ms         3  4.6090ms  988.78us  11.842ms  cudaMemcpy
                    0.45%  1.0235ms         1  1.0235ms  1.0235ms  1.0235ms  cuDeviceGetPCIBusId
                    0.23%  526.04us         3  175.35us  112.23us  208.56us  cudaFree
                    0.05%  122.05us       101  1.2080us     128ns  52.944us  cuDeviceGe